In [2]:
import serial  # 引用pySerial模組
import socket
import socketio
import numpy as np
import plotly.express as px
from time import sleep, time, ctime
from emd.sift import sift
from keyboard import is_pressed
from collections import deque

GENDER = "M"

BAUD_RATE = 115200    # 設定傳輸速率
COM_PORT = 'COM3'    # 指定通訊埠名稱
NUM_IMF = 3
CHANNEL_NUMBER = 3
SAMPLE_RATE = 500
WINDOW_SIZE = 250
NOARDUINO = True
KEY_CLASS = {0:'undefined action', 1:'up', 2:'down', 3:'left', 4:'right', 5:'quick touch'}
MAXCHARLEN = max([len(KEY_CLASS[key]) for key in KEY_CLASS])
REQUEST_CHANNEL = "inferenceRequest"
REQUEST_COOLDOWN = 0.04 #s

record_path = './data/2023_Mar_21_182116_l5m6r7_record_X.npy'
SERVER_URL = 'http://192.168.0.77:3000'

In [113]:
class remoteReceiver():
    def __init__(self, url) -> None: 
        self.__clientID = ''
        self.__sio = socketio.Client()
        self.__sio.connect(url)
        self.__sio.on('registerInfo', self.__getID)
        self.__sio.emit('signalHandlerRegister', {'time': "{:.3f}".format(time()), 'remote': True})

        self.__container = deque([], maxlen=WINDOW_SIZE)
        try:
            assert isinstance(self.__sio, socketio.Client)
            print("Connected to Socket server.")
        except:
            raise Exception("Maybe server is not online.")
        
    def __getID(self, id):
        self.__clientID = id
        self.__sio.on(id, self.__getData)
    
    def __getData(self, rcv):
        try:
            self.__container.append(rcv.split(','))
        except:
            pass
        
    def __emdSignal(self, sig):
        sig = np.array(sig).astype(np.float32).reshape(CHANNEL_NUMBER, WINDOW_SIZE).T
        ret = None
    
        for c in range(CHANNEL_NUMBER):
            raw = sig[:, c]
            imf = sift(raw, max_imfs=NUM_IMF, imf_opts={'sd_thresh': 0.1})
            
            if imf.shape[-1] < NUM_IMF:
                compensate = np.zeros((WINDOW_SIZE, NUM_IMF - imf.shape[-1]))
                imf = np.concatenate([imf, compensate], axis = 1)
            
            if not type(ret) == np.ndarray: 
                ret = imf
            else: 
                ret = np.concatenate([ret, imf], axis = 1)
              
        return ",".join(ret[np.newaxis, :].astype(np.str_).flatten().tolist())
        
    def run(self):
        try:
            escape = False
            stamp = time()
            count = 0
            while not escape:
                try:
                    if len(self.__container) == WINDOW_SIZE and time() > (stamp + REQUEST_COOLDOWN):
                        stamp = time()
                        count += 1
                        self.__sio.emit(REQUEST_CHANNEL, {'uid': self.__clientID, 'data': self.__emdSignal(self.__container), 'serial_num': count})
                        print("ID: {} send {: 5d}.".format(self.__clientID, count).ljust(22), end='\r')
                        
                except KeyboardInterrupt:
                    escape = True
                    break
                except:
                    pass
        
        except KeyboardInterrupt:
            self.__sio.disconnect()
            print('Serial disconnected.'.ljust(22))
    
class receiver():
    def __init__(self, url, port, baud_rate, record = None) -> None: 
        if isinstance(record, loader):
            self.__serial = record
        else:
            self.__serial = serial.Serial(port, baud_rate)
            
        self.__clientID = ''
        self.__serverUrl = url
        self.__sio = None
        self.__initializeSocketIOClient()

        self.__container = deque([], maxlen=WINDOW_SIZE)
        
    def __initializeSocketIOClient(self):
        self.__sio = socketio.Client(reconnection=False)
        self.__sio.connect(self.__serverUrl)
        self.__sio.on('registerInfo', self.__getID)
        self.__sio.emit('signalHandlerRegister', {'time': "{:.3f}".format(time()), 'remote': True, 'localIP': socket.gethostbyname(socket.gethostname())})
        sleep(0.1)
        try:
            assert isinstance(self.__sio, socketio.Client)
            print("Connected to Socket server with ID: {}.".format(self.__clientID))
        except:
            raise Exception("Maybe server is not online.")
        
    def __getID(self, id):
        self.__clientID = id

    def run(self):
        def clear_line(n=1):
            LINE_UP = '\033[1A'
            LINE_CLEAR = '\x1b[2K'
            for _ in range(n):
                print(LINE_UP, end=LINE_CLEAR)

        try:
            escape = False
            stamp = time()
            count = 0
            while not escape:
                c = 0
                while self.__serial.in_waiting:
                    data_raw = self.__serial.readline()
                    c += 1
                    rcv = data_raw.decode(errors='surrogateescape').rstrip()
                    try:
                        if len(rcv.split(',')) < CHANNEL_NUMBER:
                            continue
                        self.__container.append(rcv.split(',')) # a0,b0,c0, a1,b1,c1 .... an,bn,cn, <-- tackle this with rstrip() and 
                                                        # reshape with (WINDOW_SIZE, CHANNEL_SIZE) and then transpose -> (CHANNEL_SIZE, WINDOW_SIZE)
                        
                        if len(self.__container) == WINDOW_SIZE and time() > (stamp + REQUEST_COOLDOWN):
                            print("rea:", c)
                            stamp = time()
                            count += 1
                            # self.__sio.emit(config.REQUEST_CHANNEL, {'uid': self.__clientID, 'data': self.__emdSignal(self.__container), 'serial_num': count})
                            self.__sio.emit(REQUEST_CHANNEL, {'uid': self.__clientID,
                                                                'data': ",".join(np.array(self.__container).flatten().tolist()),
                                                                'serial_num': count})
                            print("ID: {} send {}.".format(self.__clientID, count))
                            clear_line()

                    except KeyboardInterrupt:
                        escape = True
                        break
                    except:
                        pass
                            
            raise KeyboardInterrupt

        except KeyboardInterrupt:
            self.__serial.close() 
            self.__sio.disconnect()
            print('Serial disconnected.'.ljust((len(self.__clientID) + int(np.log10(count)) + 12)))
            
    def __emdSignal(self, sig):
        sig = np.array(sig).astype(np.float16).reshape(WINDOW_SIZE, CHANNEL_NUMBER).T
        ret = None

        for c in range(CHANNEL_NUMBER):
            raw = sig[:, c]
            imf = sift(raw, max_imfs=NUM_IMF, imf_opts={'sd_thresh': 0.1})
            
            if imf.shape[-1] < NUM_IMF:
                compensate = np.zeros((WINDOW_SIZE, NUM_IMF - imf.shape[-1]))
                imf = np.concatenate([imf, compensate], axis = 1)
            
            if not type(ret) == np.ndarray: 
                ret = imf
            else: 
                ret = np.concatenate([ret, imf], axis = 1)
                
        return ",".join(ret[np.newaxis, :].astype(np.str_).flatten().tolist())
            
class recorder():
    def __init__(self, port, baud_rate, record_name = '', record_path = './data', fakeSignal = None) -> None: 
        if isinstance(fakeSignal, loader):
            self.__serial = fakeSignal
        else:
            self.__serial = serial.Serial(port, baud_rate)
        self.X = None
        self.y = []
        if record_name == '':
            self.__record_path = None
        else:
            self.__record_path = record_path + '/' + record_name
    
    def run(self):
        try:
            escape = False
            start = time()
            while not escape:
                while self.__serial.in_waiting:
                    data_raw = self.__serial.readline()  
                    rcv = data_raw.decode(errors='surrogateescape').rstrip() # decode with UTF-8
                    try:
                        #Hand-input labeling
                        label = 0
                        if is_pressed('8'):
                            label = 1
                        elif is_pressed('5'):
                            label = 2
                        elif is_pressed('4'):
                            label = 3
                        elif is_pressed('6'):
                            label = 4
                        elif is_pressed('1'):
                            label = 5
                        elif is_pressed('escape'):
                            raise KeyboardInterrupt

                        data = np.array(rcv.split(',')).astype(np.float32)
                        if data.shape[0] < CHANNEL_NUMBER:
                            continue
                        
                        self.y.append(label)
                        print(("{:.2f}".format(time() - start) + ' ' + KEY_CLASS[self.y[-1]]).ljust(len(KEY_CLASS[0]) + len(str(int(time()))) + 3), end='\r')
                        
                        #Store recieved data
                        if isinstance(self.X, np.ndarray):
                            self.X = np.concatenate([self.X, data[np.newaxis, :].copy()], axis=0)
                        else:
                            self.X = data[np.newaxis, :].copy()              
                            
                    except KeyboardInterrupt:
                        escape = True
                        break
                    except:
                        pass
                            
            raise KeyboardInterrupt

        except KeyboardInterrupt:
            if isinstance(self.__record_path, str):
                #Save recorded data
                np.save(self.__record_path + '_X', self.X)
                print("Saved file {}.".format(self.__record_path + '_X'))

                np.save(self.__record_path + '_y', np.array(self.y).astype(np.int32))
                print("Saved file {}.".format(self.__record_path + '_y'))

            self.__serial.close()
            print('Serial disconnected.'.ljust(50))
            

class loader():
    def __init__(self, path) -> None:
        self.data = np.load(path)
        self.in_waiting = True
        self.__i = -1
        
    def readline(self):
        try:
            self.__i += 1
            return ",".join(["%.0f" %x for x in self.data[100 + self.__i, :].tolist()]).encode(encoding='UTF-8')
        except KeyboardInterrupt:
            self.in_waiting = False
        except:
            if self.__i + 100 >= self.data.shape[0]:
                self.__i = -1
            return ",".join(["%.0f" %x for x in self.data[101 + self.__i, :].tolist()]).encode(encoding='UTF-8')
    
    def close(self):
        pass

In [106]:
# (Remote) Recieve real-time signal and send to inference server
signal = remoteReceiver(SERVER_URL)

Connected to Socket server.


In [114]:
# Recieve real-time signal and send to inference server
if NOARDUINO:
    fake = loader(record_path)
    signal = receiver(SERVER_URL, COM_PORT, BAUD_RATE, fake)
else:
    signal = receiver(SERVER_URL, COM_PORT, BAUD_RATE)

Connected to Socket server with ID: 192.168.0.77.


In [ ]:
# Recieve, record and label real-time signal
if NOARDUINO:
    signal = recorder(COM_PORT, BAUD_RATE, fakeSignal = loader(record_path))
else:
    rtime = (lambda x:x[-1] + '_' + x[-4] + '_' + x[-3] + '_' + "".join(x[-2].split(":")))(ctime().split(" "))
    record_name = rtime + '_l5m6r7_record_Empty'
    print(record_name)
    signal = recorder(COM_PORT, BAUD_RATE, record_name)

In [115]:
signal.run()

rea: 250
ID: 192.168.0.77 send 1.
send: -4,0,-4,0,5,-2,3,9,5,2,7,9,0,-1,3,0,-7,-3,0,-6,-6,0,-5,-5,0,-3,-2,0,4,3,3,7,7,4,3,4,1,1,1,-2,0,0,-2,-4,-3,-1,-3,-5,0,0,-3,0,8,1,1,8,4,-1,-6,2,-1,-10,0,-1,-4,-4,0,-1,-6,0,0,0,0,4,6,0,7,5,0,5,0,0,0,-3,0,0,-4,0,1,-4,0,0,-1,1,-4,0,0,-7,0,-2,-6,3,0,0,1,2,1,0,1,0,0,-1,2,1,-2,4,1,-2,5,1,0,3,2,1,-2,0,3,-5,-1,2,-3,0,0,4,0,-1,9,-3,2,-3,0,3,-15,1,-4,-8,0,-10,2,-2,-7,5,-1,2,4,0,6,3,1,4,6,3,1,8,3,1,1,5,4,-8,1,1,-7,-5,0,-3,-9,2,-1,-6,-4,0,1,-5,-1,5,0,0,2,-1,2,0,-3,5,0,-2,2,0,-1,-1,0,2,-3,0,0,-1,0,-2,-1,2,-2,-1,3,0,0,0,2,2,-2,2,2,-3,2,0,-5,1,1,-2,1,0,4,-1,-2,4,0,0,0,1,0,-1,2,1,-1,2,2,0,1,0,3,0,0,2,-4,1,2,-3,4,0,0,2,-1,-1,-9,-5,-4,-11,-8,-1,0,-2,1,4,4,1,5,5,2,6,3,3,3,0,0,-5,-1,-1,-5,-2,2,1,-4,3,4,-2,2,4,0,4,0,0,3,-3,1,-3,-2,1,-5,-1,3,-5,0,6,-7,-1,2,-3,-2,-2,4,0,-1,2,1,-2,-3,0,-3,-1,-5,-4,3,-8,-4,2,0,0,0,6,5,-2,4,6,-2,4,3,-2,3,1,-2,5,0,1,4,-4,4,0,-4,4,-2,0,3,-4,3,1,-4,3,-1,-2,0,-3,-2,0,0,-4,1,1,-1,-1,0,4,-4,0,6,-2,2,4,2,2,0,3,0,-2,-1,-1,-2,-7,0,-1,-6,-2,-1,-1,0,0

In [ ]:
px.line(signal.X[100:])

In [ ]:
px.line(signal.y[100:])

In [ ]:
np.save('./data/2023_Mar_21_190722_l5m6r7_record_X', signal.X[:20000])
np.save('./data/2023_Mar_21_190722_l5m6r7_record_y', np.array(signal.y[:20000]).astype(np.int32))

In [ ]:
# import holoviews as hv
# from holoviews.streams import Buffer
# hv.extension('bokeh')
# renderer = hv.renderer('bokeh')

# class RecordPlotter():
#     def __init__(self, record_name = '', record_path = './data/'):
#         self.__buffers = [Buffer(np.zeros((0, 2)), length=int(WINDOW_SIZE//2)) for i in range(CHANNEL_NUMBER)]
#         self.__X = np.load(record_path + record_name + '_X.npy')
#         self.__y = np.load(record_path + record_name + '_y.npy')

#     def buildPlot(self):
#         plt = hv.DynamicMap(hv.Curve, streams=[self.__buffers[0]]).relabel('0')
#         for i in range(1, CHANNEL_NUMBER):
#             plt *= hv.DynamicMap(hv.Curve, streams=[self.__buffers[i]]).relabel(str(i))
#         return plt.opts(padding=0.1, width=600)

#     def streaming(self):
#         escape = False
#         start = time()
#         while not escape:
#             try:
#                 for i in range(self.__X.shape[0]):
#                     for j in range(CHANNEL_NUMBER):
#                         self.__buffers[j].send(np.array([[i, self.__X[i, j]]]))
#                     print(("{:.2f} ".format(time() - start) + KEY_CLASS[self.__y[i]]).ljust(MAXCHARLEN + len(str(int(time()))) + 3), end='\r')
#                     sleep(1 / SAMPLE_RATE)
                    
#             except KeyboardInterrupt:
#                 escape = True
#                 for i in range(CHANNEL_NUMBER):
#                     self.__buffers[i].clear()
#                 break